##### Copyright 2020 Die TensorFlow-Autoren.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Erweiterte automatische Differenzierung

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/advanced_autodiff"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> Ansicht auf TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/advanced_autodiff.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> In Google Colab ausführen</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/advanced_autodiff.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Quelle auf GitHub anzeigen</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/advanced_autodiff.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png"> Notizbuch herunterladen</a></td>
</table>

Die [automatische Differenzierungsanleitung](autodiff.ipynb) enthält alles, was zur Berechnung von Verläufen erforderlich ist. Dieser Leitfaden konzentriert sich auf tiefere, weniger verbreitete Funktionen der `tf.GradientTape` API.

## Konfiguration

In [ ]:
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['figure.figsize'] = (8, 6)

## Steuerung der Gradientenaufzeichnung

In der [automatischen Differenzierungsanleitung haben](autodiff.ipynb) Sie gesehen, wie Sie steuern können, welche Variablen und Tensoren vom Band überwacht werden, während Sie die Gradientenberechnung erstellen.

Das Band verfügt auch über Methoden zum Manipulieren der Aufnahme.

Wenn Sie die Aufzeichnung von Verläufen beenden möchten, können Sie `GradientTape.stop_recording()` um die Aufnahme vorübergehend auszusetzen.

Dies kann nützlich sein, um den Overhead zu reduzieren, wenn Sie eine komplizierte Operation in der Mitte Ihres Modells nicht unterscheiden möchten. Dies kann die Berechnung einer Metrik oder eines Zwischenergebnisses umfassen:

In [ ]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)

with tf.GradientTape() as t:
  x_sq = x * x
  with t.stop_recording():
    y_sq = y * y
  z = x_sq + y_sq

grad = t.gradient(z, {'x': x, 'y': y})

print('dz/dx:', grad['x'])  # 2*x => 4
print('dz/dy:', grad['y'])

Wenn Sie ganz von vorne beginnen möchten, verwenden Sie `reset()` . Das einfache Verlassen des Verlaufsbandblocks und das Neustarten ist normalerweise einfacher zu lesen. Sie können jedoch das `reset` wenn das Verlassen des Bandblocks schwierig oder unmöglich ist.

In [ ]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)
reset = True

with tf.GradientTape() as t:
  y_sq = y * y
  if reset:
    # Throw out all the tape recorded so far
    t.reset()
  z = x * x + y_sq

grad = t.gradient(z, {'x': x, 'y': y})

print('dz/dx:', grad['x'])  # 2*x => 4
print('dz/dy:', grad['y'])

## Steigung stoppen

In contrast to the global tape controls above, the `tf.stop_gradient` function is much more precise. It can be used to stop gradients from flowing along a particular path, without needing access to the tape itself:

In [ ]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)

with tf.GradientTape() as t:
  y_sq = y**2
  z = x**2 + tf.stop_gradient(y_sq)

grad = t.gradient(z, {'x': x, 'y': y})

print('dz/dx:', grad['x'])  # 2*x => 4
print('dz/dy:', grad['y'])

## Benutzerdefinierte Farbverläufe

In einigen Fällen möchten Sie möglicherweise genau steuern, wie Verläufe berechnet werden, anstatt die Standardeinstellung zu verwenden. Diese Situationen umfassen:

- Es gibt keinen definierten Farbverlauf für eine neue Operation, die Sie schreiben.
- Die Standardberechnungen sind numerisch instabil.
- Sie möchten eine teure Berechnung aus dem Vorwärtsdurchlauf zwischenspeichern.
- Sie möchten einen Wert ändern (z. B. mit: `tf.clip_by_value` , `tf.math.round` ), ohne den Verlauf zu ändern.

For writing a new op, you can use `tf.RegisterGradient` to set up your own. See that page for details. (Note that the gradient registry is global, so change it with caution.)

In den letzten drei Fällen können Sie `tf.custom_gradient` .


Here is an example that applies `tf.clip_by_norm` to the intermediate gradient.

In [ ]:
# Establish an identity operation, but clip during the gradient pass
@tf.custom_gradient
def clip_gradients(y):
  def backward(dy):
    return tf.clip_by_norm(dy, 0.5)
  return y, backward

v = tf.Variable(2.0)
with tf.GradientTape() as t:
  output = clip_gradients(v * v)
print(t.gradient(output, v))  # calls "backward", which clips 4 to 2


See the `tf.custom_gradient` decorator for more details.

## Mehrere Bänder

Mehrere Bänder interagieren nahtlos. Zum Beispiel beobachtet hier jedes Band einen anderen Satz von Tensoren:

In [ ]:
x0 = tf.constant(0.0)
x1 = tf.constant(0.0)

with tf.GradientTape() as tape0, tf.GradientTape() as tape1:
  tape0.watch(x0)
  tape1.watch(x1)

  y0 = tf.math.sin(x0)
  y1 = tf.nn.sigmoid(x1)

  y = y0 + y1

  ys = tf.reduce_sum(y)

In [ ]:
tape0.gradient(ys, x0).numpy()   # cos(x) => 1.0

In [ ]:
tape1.gradient(ys, x1).numpy()   # sigmoid(x1)*(1-sigmoid(x1)) => 0.25

### Gradienten höherer Ordnung

Vorgänge im `GradientTape` Kontextmanager werden zur automatischen Differenzierung aufgezeichnet. Wenn in diesem Zusammenhang Gradienten berechnet werden, wird auch die Gradientenberechnung aufgezeichnet. Infolgedessen funktioniert genau dieselbe API auch für Gradienten höherer Ordnung. Beispielsweise:

In [ ]:
x = tf.Variable(1.0)  # Create a Tensorflow variable initialized to 1.0

with tf.GradientTape() as t2:
  with tf.GradientTape() as t1:
    y = x * x * x

  # Compute the gradient inside the outer `t2` context manager
  # which means the gradient computation is differentiable as well.
  dy_dx = t1.gradient(y, x)
d2y_dx2 = t2.gradient(dy_dx, x)

print('dy_dx:', dy_dx.numpy())  # 3 * x**2 => 3.0
print('d2y_dx2:', d2y_dx2.numpy())  # 6 * x => 6.0

Während dies die zweite Ableitung einer *Skalarfunktion* ergibt, wird dieses Muster nicht verallgemeinert, um eine hessische Matrix zu erzeugen, da `GradientTape.gradient` nur den Gradienten eines Skalars berechnet. Informationen zum Konstruieren eines Hessischen finden Sie im [hessischen Beispiel](#hessian) im [Abschnitt Jacobian](#jacobians) .

"Verschachtelte Aufrufe von `GradientTape.gradient` " ist ein gutes Muster, wenn Sie einen Skalar aus einem Gradienten berechnen. Der resultierende Skalar fungiert dann als Quelle für eine zweite Gradientenberechnung, wie im folgenden Beispiel.


#### Beispiel: Regularisierung des Eingabegradienten

Viele Modelle sind anfällig für "kontroverse Beispiele". Diese Sammlung von Techniken ändert die Eingabe des Modells, um die Ausgabe des Modells zu verwirren. Die [einfachste Implementierung](https://www.tensorflow.org/tutorials/generative/adversarial_fgsm) erfolgt in einem einzigen Schritt entlang des Gradienten der Ausgabe in Bezug auf die Eingabe. der "Eingangsgradient".

Eine Technik zur Erhöhung der Robustheit gegenüber gegnerischen Beispielen ist die [Regularisierung](https://arxiv.org/abs/1905.11468) des Eingabegradienten, mit der versucht wird, die Größe des Eingabegradienten zu minimieren. Wenn der Eingangsgradient klein ist, sollte auch die Änderung des Ausgangs gering sein.

Im Folgenden finden Sie eine naive Implementierung der Regularisierung des Eingabegradienten. Die Implementierung ist:

1. Berechnen Sie den Gradienten der Ausgabe in Bezug auf die Eingabe mit einem inneren Band.
2. Berechnen Sie die Größe dieses Eingangsgradienten.
3. Berechnen Sie den Gradienten dieser Größe in Bezug auf das Modell.

In [ ]:
x = tf.random.normal([7, 5])

layer = tf.keras.layers.Dense(10, activation=tf.nn.relu)

In [ ]:
with tf.GradientTape() as t2:
  # The inner tape only takes the gradient with respect to the input,
  # not the variables.
  with tf.GradientTape(watch_accessed_variables=False) as t1:
    t1.watch(x)
    y = layer(x)
    out = tf.reduce_sum(layer(x)**2)
  # 1. Calculate the input gradient.
  g1 = t1.gradient(out, x)
  # 2. Calculate the magnitude of the input gradient.
  g1_mag = tf.norm(g1)

# 3. Calculate the gradient of the magnitude with respect to the model.
dg1_mag = t2.gradient(g1_mag, layer.trainable_variables)

In [ ]:
[var.shape for var in dg1_mag]

## Jacobianer


In allen vorherigen Beispielen wurden die Gradienten eines skalaren Ziels in Bezug auf einen oder mehrere Quellentensoren verwendet.

Die [Jacobi-Matrix](https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant) repräsentiert die Gradienten einer Vektorwertfunktion. Jede Zeile enthält den Gradienten eines der Elemente des Vektors.

Mit der `GradientTape.jacobian` Methode können Sie eine Jacobian-Matrix effizient berechnen.

Beachten Sie, dass:

- Wie `gradient` : Das `sources` kann ein Tensor oder ein Container mit Tensoren sein.
- Im Gegensatz zu `gradient` : Der `target` Tensor ein einzelner Tensor sein muss.

### Skalare Quelle

Als erstes Beispiel ist hier der Jacobi eines Vektorziels in Bezug auf eine Skalarquelle.

In [ ]:
x = tf.linspace(-10.0, 10.0, 200+1)
delta = tf.Variable(0.0)

with tf.GradientTape() as tape:
  y = tf.nn.sigmoid(x+delta)

dy_dx = tape.jacobian(y, delta)

Wenn Sie den Jacobi in Bezug auf einen Skalar nehmen, hat das Ergebnis die Form des **Ziels** und gibt den Gradienten jedes Elements in Bezug auf die Quelle an:

In [ ]:
print(y.shape)
print(dy_dx.shape)

In [ ]:
plt.plot(x.numpy(), y, label='y')
plt.plot(x.numpy(), dy_dx, label='dy/dx')
plt.legend()
_ = plt.xlabel('x')

### Tensorquelle

Unabhängig davon, ob die Eingabe skalar oder tensorisch ist, berechnet `GradientTape.jacobian` effizient den Gradienten jedes Elements der Quelle in Bezug auf jedes Element des Ziels (der Ziele).

Zum Beispiel hat die Ausgabe dieser Ebene eine Form von `(10, 7)` :

In [ ]:
x = tf.random.normal([7, 5])
layer = tf.keras.layers.Dense(10, activation=tf.nn.relu)

with tf.GradientTape(persistent=True) as tape:
  y = layer(x)

y.shape

Und die Kernelform der Ebene ist `(5, 10)` :

In [ ]:
layer.kernel.shape

Die Form des Jacobi der Ausgabe in Bezug auf den Kernel sind die beiden miteinander verketteten Formen:

In [ ]:
j = tape.jacobian(y, layer.kernel)
j.shape

Wenn Sie über die Dimensionen des Ziels summieren, bleibt der Gradient der Summe übrig, der von `GradientTape.gradient` berechnet worden wäre:

In [ ]:
g = tape.gradient(y, layer.kernel)
print('g.shape:', g.shape)

j_sum = tf.reduce_sum(j, axis=[0, 1])
delta = tf.reduce_max(abs(g - j_sum)).numpy()
assert delta < 1e-3
print('delta:', delta)

<a id="hessian"> </a>

#### Beispiel: Hessisch

While `tf.GradientTape` doesn't give an explicit method for constructing a Hessian matrix it's possible to build one using the `GradientTape.jacobian` method.

Hinweis: Die hessische Matrix enthält `N**2` Parameter. Aus diesem und anderen Gründen ist es für die meisten Modelle nicht praktikabel. Dieses Beispiel dient eher als Demonstration der Verwendung der `GradientTape.jacobian` Methode und ist keine Bestätigung der direkten hessischen Optimierung. Ein Hessisches Vektorprodukt kann [mit verschachtelten Bändern effizient berechnet werden](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/eager/benchmarks/resnet50/hvp_test.py) und ist ein viel effizienterer Ansatz zur Optimierung zweiter Ordnung.


In [ ]:
x = tf.random.normal([7, 5])
layer1 = tf.keras.layers.Dense(8, activation=tf.nn.relu)
layer2 = tf.keras.layers.Dense(6, activation=tf.nn.relu)

with tf.GradientTape() as t2:
  with tf.GradientTape() as t1:
    x = layer1(x)
    x = layer2(x)
    loss = tf.reduce_mean(x**2)

  g = t1.gradient(loss, layer1.kernel)

h = t2.jacobian(g, layer1.kernel)

In [ ]:
print(f'layer.kernel.shape: {layer1.kernel.shape}')
print(f'h.shape: {h.shape}')

Um dieses Hessische für einen Newtonschen Methodenschritt zu verwenden, würden Sie zuerst seine Achsen in eine Matrix reduzieren und den Gradienten in einen Vektor reduzieren:

In [ ]:
n_params = tf.reduce_prod(layer1.kernel.shape)

g_vec = tf.reshape(g, [n_params, 1])
h_mat = tf.reshape(h, [n_params, n_params])

Die hessische Matrix sollte symmetrisch sein:

In [ ]:
def imshow_zero_center(image, **kwargs):
  lim = tf.reduce_max(abs(image))
  plt.imshow(image, vmin=-lim, vmax=lim, cmap='seismic', **kwargs)
  plt.colorbar()

In [ ]:
imshow_zero_center(h_mat)

Der Schritt zur Aktualisierung der Newton-Methode ist unten dargestellt.

In [ ]:
eps = 1e-3
eye_eps = tf.eye(h_mat.shape[0])*eps

Hinweis: [Invertieren Sie die Matrix nicht](https://www.johndcook.com/blog/2010/01/19/dont-invert-that-matrix/) .

In [ ]:
# X(k+1) = X(k) - (∇²f(X(k)))^-1 @ ∇f(X(k))
# h_mat = ∇²f(X(k))
# g_vec = ∇f(X(k))
update = tf.linalg.solve(h_mat + eye_eps, g_vec)

# Reshape the update and apply it to the variable.
_ = layer1.kernel.assign_sub(tf.reshape(update, layer1.kernel.shape))

While this is relatively simple for a single `tf.Variable`, applying this to a non-trivial model would require careful concatenation and slicing to produce a full Hessian across multiple variables.

### Batch Jacobian

In einigen Fällen möchten Sie den Jacobi jedes Zielstapels in Bezug auf einen Quellenstapel nehmen, wobei die Jacobi für jedes Ziel-Quell-Paar unabhängig sind.

Hier ist beispielsweise die Eingabe `x` geformt `(batch, ins)` und die Ausgabe `y` geformt `(batch, outs)` .


In [ ]:
x = tf.random.normal([7, 5])

layer1 = tf.keras.layers.Dense(8, activation=tf.nn.elu)
layer2 = tf.keras.layers.Dense(6, activation=tf.nn.elu)

with tf.GradientTape(persistent=True, watch_accessed_variables=False) as tape:
  tape.watch(x)
  y = layer1(x)
  y = layer2(y)

y.shape

Der vollständige Jacobi von `y` in Bezug auf `x` hat die Form `(batch, ins, batch, outs)` , auch wenn Sie nur möchten `(batch, ins, outs)` .

In [ ]:
j = tape.jacobian(y, x)
j.shape

Wenn die Verläufe jedes Elements im Stapel unabhängig sind, ist jede `(batch, batch)` Schicht dieses Tensors eine diagonale Matrix:

In [ ]:
imshow_zero_center(j[:, 0, :, 0])
_ = plt.title('A (batch, batch) slice')

In [ ]:
def plot_as_patches(j):
  # Reorder axes so the diagonals will each form a contiguous patch.
  j = tf.transpose(j, [1, 0, 3, 2])
  # Pad in between each patch.
  lim = tf.reduce_max(abs(j))
  j = tf.pad(j, [[0, 0], [1, 1], [0, 0], [1, 1]],
             constant_values=-lim)
  # Reshape to form a single image.
  s = j.shape
  j = tf.reshape(j, [s[0]*s[1], s[2]*s[3]])
  imshow_zero_center(j, extent=[-0.5, s[2]-0.5, s[0]-0.5, -0.5])

plot_as_patches(j)
_ = plt.title('All (batch, batch) slices are diagonal')

To get the desired result you can sum over the duplicate `batch` dimension, or else select the diagonals using `tf.einsum`.


In [ ]:
j_sum = tf.reduce_sum(j, axis=2)
print(j_sum.shape)
j_select = tf.einsum('bxby->bxy', j)
print(j_select.shape)

Es wäre viel effizienter, die Berechnung zunächst ohne die zusätzliche Dimension durchzuführen. Die `GradientTape.batch_jacobian` Methode macht genau das.

In [ ]:
jb = tape.batch_jacobian(y, x)
jb.shape

In [ ]:
error = tf.reduce_max(abs(jb - j_sum))
assert error < 1e-3
print(error.numpy())

Caution: `GradientTape.batch_jacobian` only verifies that the first dimension of the source and target match. It doesn't check that the gradients are actually independent. It's up to the user to ensure they only use `batch_jacobian` where it makes sense. For example adding a `layers.BatchNormalization` destroys the independence, since it normalizes across the `batch` dimension:

In [ ]:
x = tf.random.normal([7, 5])

layer1 = tf.keras.layers.Dense(8, activation=tf.nn.elu)
bn = tf.keras.layers.BatchNormalization()
layer2 = tf.keras.layers.Dense(6, activation=tf.nn.elu)

with tf.GradientTape(persistent=True, watch_accessed_variables=False) as tape:
  tape.watch(x)
  y = layer1(x)
  y = bn(y, training=True)
  y = layer2(y)

j = tape.jacobian(y, x)
print(f'j.shape: {j.shape}')

In [ ]:
plot_as_patches(j)

_ = plt.title('These slices are not diagonal')
_ = plt.xlabel("Don't use `batch_jacobian`")

In diesem Fall wird `batch_jacobian` immer noch ausgeführt und gibt *etwas* mit der erwarteten Form zurück, aber der Inhalt hat eine unklare Bedeutung.

In [ ]:
jb = tape.batch_jacobian(y, x)
print(f'jb.shape: {jb.shape}')